In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Parameter
import torch.nn.functional as F
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.preprocessing import MinMaxScaler

In [13]:
num_nodes = 10

train_rate = 0.5
seq_len = 20
output_dim = pred_len = 10
batch_size = 32
lr = 0.001
training_epoch = 1001
validation_rate = 0.1
l2_coeff = 0.0001


#for ILI Region data
adj_pearson = pd.read_csv('input_data/Region_adj_geo1.csv', header=None)
adj = np.mat(adj_pearson)

#region features(data)
data = pd.read_csv('input_data/ILI_region_feature_1997-2020.csv')
data = np.mat(data, dtype=np.float32)

scaler = MinMaxScaler()
scaler.fit(data[0:int(len(data)/2)])   #절반까지(train data)에 대해서만 fit
data = scaler.transform(data)

print(data[-5:])


[[1.82012    3.1085544  1.6115223  0.6289664  0.4157108  0.7908482
  0.28640518 0.37363324 0.31908202 2.917647  ]
 [1.3491006  1.869935   1.2565175  0.44287363 0.33733055 0.5245536
  0.18484686 0.25835678 0.23137377 2.1294117 ]
 [1.254497   1.1125563  1.0218861  0.33155304 0.28832117 0.38214287
  0.13272434 0.17213371 0.18170387 1.4282353 ]
 [0.8554297  0.6910956  0.96202123 0.2925961  0.287626   0.3502232
  0.13057497 0.16526085 0.1615844  1.435294  ]
 [0.68820786 0.48724365 0.8052784  0.2448424  0.2716371  0.28191966
  0.1144546  0.12527335 0.14806664 1.2917646 ]]


In [49]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.get_device_name()

(device(type='cuda', index=0), 'NVIDIA GeForce RTX 2080 SUPER')

In [14]:
time_len =data.shape[0]     #총 시간
num_nodes = data.shape[1]   #지역 node 갯수
print(time_len, num_nodes)

1179 10


In [57]:
def preprocess_data(data, time_len, train_rate, validation_rate, seq_len, pred_len):
    """
    하나의 node에 대해서 seq_len=20 짜리의 데이터를 취한 것을, 
    모든 node=10 에 대해서 쌓아서, 
    하나의 time_step에 대한 data를 만들고,
    모든 time_step에 대해서 쌓은 크기
    의 data를 floattensor로 반환하는 함수
    """
    train_size = int(time_len * train_rate)
    validation_size = int(time_len * validation_rate)
    print(f"train_size : {train_size}, validation_size : {validation_size}")
    train_data = data[0:train_size]
    print(f"train_data shape : {train_data.shape}")
    validation_data = data[train_size - seq_len : train_size + validation_size] #train data 바로 다음 시점부터 validation 시작
    test_data = data[train_size + validation_size - seq_len : time_len]
    print(f"test_data shape : {test_data.shape}")
    
    X_train, y_train, X_test, y_test = [], [], [], []
    
    for i in range(len(train_data) - seq_len - pred_len):
        
        temp = train_data[i:i + seq_len + pred_len]
        
        X_train.append(np.transpose(temp[:seq_len]))    #여기선 transpose인데
        y_train.append(temp[seq_len + pred_len - 1])    #여기선 왜 단일 값? -> 
        
    for i in range(len(test_data) - seq_len - pred_len):
        
        temp = test_data[i:i + seq_len + pred_len]
        
        X_test.append(np.transpose(temp[:seq_len]))
        y_test.append(temp[seq_len + pred_len - 1])
        
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    print(f"\nX_train : {X_train.shape} | y_train : {y_train.shape} \nX_test : {X_test.shape} | y_test : {y_test.shape}")
    
    return torch.FloatTensor(X_train).to(device), torch.FloatTensor(y_train).to(device), torch.FloatTensor(X_test).to(device), torch.FloatTensor(y_test).to(device)

In [56]:
X_train, y_train, X_test, y_test = preprocess_data(data, time_len, train_rate, validation_rate, seq_len, pred_len)
#print(X_train.dtype, y_train.dtype)


train_size : 589, validation_size : 117
train_data shape : (589, 10)
test_data shape : (493, 10)

X_train : (559, 10, 20) | y_train : (559, 10) 
X_test : (463, 10, 20) | y_test : (463, 10)


In [29]:
training_data_count = len(X_train)
print(training_data_count, X_train.shape)

559 (559, 10, 20)


In [70]:
train = torch.utils.data.TensorDataset(X_train, y_train)
test = torch.utils.data.TensorDataset(X_test, y_test)

trainloader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
testloader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

In [77]:
rnn_hidden_size = 12    #for dim check 12
rnn_num_layers = 1

In [78]:
'''
#model 앞부분에 node별로 temporal dependency를 추출하기 위한 RNN

class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, device):
        super().__init__()
        self.device = device
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size = self.input_size, hidden_size = self.hidden_size, num_layers = self.num_layers, batch_first = True)
        #여기서는 RNN output을 그대로 사용할 것 이기 때문에 마지막 sequential layer가 필요 없을 것 으로 생각
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device)      #나중에 node 별로 모델 새로 선언 필요
        #initial hidden state 여기서는 zeros, 나중에는 normalized values 시도 | x.size()[0] == batch_size, hidden_size 정해줘야함
        _, h0 = self.rnn(x, h0)       #***우리는 여기서 hidden_state가 필요함! 나중에 모든 node에 대해서 hidden_state들을 쌓아야됨***
        #문제는, 여기서의 output값이 필요한지는 확실치 않음 - 이 RNN으로 값을 예측하려는 것이 아니기 때문, 아마 hidden 만 쓸 것으로 생각
        
        return h0
'''

In [91]:


#model 앞부분에 node별로 temporal dependency를 추출하기 위한 RNN

class cola_gnn(nn.Module):
    
    def __init__(self, input_size, rnn_hidden_size, sequence_length, rnn_num_layers, conv_channel_num=8, device=device):
        super().__init__()
        self.device = device            #cuda else cpu
        self.input_size = input_size    #10(node)*20(seq_length) tensor
        self.rnn_hidden_size = rnn_hidden_size
        self.sequence_length = sequence_length  #seq_length
        self.rnn_num_layers = rnn_num_layers    #
        self.conv_channel_num = conv_channel_num
        self.RNN_modules = {
            'RNN_0' : nn.RNN(input_size = self.sequence_length, hidden_size = self.rnn_hidden_size, num_layers = self.rnn_num_layers, batch_first = True, nonlinearity = 'tanh', device=self.device),
            'RNN_1' : nn.RNN(input_size = self.sequence_length, hidden_size = self.rnn_hidden_size, num_layers = self.rnn_num_layers, batch_first = True, nonlinearity = 'tanh', device=self.device),
            'RNN_2' : nn.RNN(input_size = self.sequence_length, hidden_size = self.rnn_hidden_size, num_layers = self.rnn_num_layers, batch_first = True, nonlinearity = 'tanh', device=self.device),
            'RNN_3' : nn.RNN(input_size = self.sequence_length, hidden_size = self.rnn_hidden_size, num_layers = self.rnn_num_layers, batch_first = True, nonlinearity = 'tanh', device=self.device),
            'RNN_4' : nn.RNN(input_size = self.sequence_length, hidden_size = self.rnn_hidden_size, num_layers = self.rnn_num_layers, batch_first = True, nonlinearity = 'tanh', device=self.device),
            'RNN_5' : nn.RNN(input_size = self.sequence_length, hidden_size = self.rnn_hidden_size, num_layers = self.rnn_num_layers, batch_first = True, nonlinearity = 'tanh', device=self.device),
            'RNN_6' : nn.RNN(input_size = self.sequence_length, hidden_size = self.rnn_hidden_size, num_layers = self.rnn_num_layers, batch_first = True, nonlinearity = 'tanh', device=self.device),
            'RNN_7' : nn.RNN(input_size = self.sequence_length, hidden_size = self.rnn_hidden_size, num_layers = self.rnn_num_layers, batch_first = True, nonlinearity = 'tanh', device=self.device),
            'RNN_8' : nn.RNN(input_size = self.sequence_length, hidden_size = self.rnn_hidden_size, num_layers = self.rnn_num_layers, batch_first = True, nonlinearity = 'tanh', device=self.device),
            'RNN_9' : nn.RNN(input_size = self.sequence_length, hidden_size = self.rnn_hidden_size, num_layers = self.rnn_num_layers, batch_first = True, nonlinearity = 'tanh', device=self.device)  
        }
        
        self.Dil_Conv_short_modules = {
            'Dil_Conv_0' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length, device=self.device),
            'Dil_Conv_1' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length, device=self.device),
            'Dil_Conv_2' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length, device=self.device),
            'Dil_Conv_3' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length, device=self.device),
            'Dil_Conv_4' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length, device=self.device),
            'Dil_Conv_5' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length, device=self.device),
            'Dil_Conv_6' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length, device=self.device),
            'Dil_Conv_7' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length, device=self.device),
            'Dil_Conv_8' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length, device=self.device),
            'Dil_Conv_9' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length, device=self.device)
            #'Dil_Conv_long' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length - self.input_size, dilation=2)
        }
        self.Dil_Conv_long_modules = {
            #'Dil_Conv_short' : nn.Conv1d(1, self.conv_channel_num, self.sequence_length),
            'Dil_Conv_0' : nn.Conv1d(1, self.conv_channel_num, int(self.sequence_length/2), dilation=2, device=self.device),
            'Dil_Conv_1' : nn.Conv1d(1, self.conv_channel_num, int(self.sequence_length/2), dilation=2, device=self.device),
            'Dil_Conv_2' : nn.Conv1d(1, self.conv_channel_num, int(self.sequence_length/2), dilation=2, device=self.device),
            'Dil_Conv_3' : nn.Conv1d(1, self.conv_channel_num, int(self.sequence_length/2), dilation=2, device=self.device),
            'Dil_Conv_4' : nn.Conv1d(1, self.conv_channel_num, int(self.sequence_length/2), dilation=2, device=self.device),
            'Dil_Conv_5' : nn.Conv1d(1, self.conv_channel_num, int(self.sequence_length/2), dilation=2, device=self.device),
            'Dil_Conv_6' : nn.Conv1d(1, self.conv_channel_num, int(self.sequence_length/2), dilation=2, device=self.device),
            'Dil_Conv_7' : nn.Conv1d(1, self.conv_channel_num, int(self.sequence_length/2), dilation=2, device=self.device),
            'Dil_Conv_8' : nn.Conv1d(1, self.conv_channel_num, int(self.sequence_length/2), dilation=2, device=self.device),
            'Dil_Conv_9' : nn.Conv1d(1, self.conv_channel_num, int(self.sequence_length/2), dilation=2, device=self.device)
        }
        #여기 지금 길이 (8,2)라는데 수정필요함!!!!!!!!!!!!!
        
        #여기서는 RNN hidden state를 그대로 사용할 것 이기 때문에 마지막 sequential layer가 필요 없음
        
        
        #RNN 선언하고
        #~Dilated Conv 선언하고
        #Loc-Aware-Attn Matrix 만들고
        #graph message passing 하고
        #output layer 제작하고
        #backprop은 나중에 최적화
        
    def forward(self, x):
        '''
        ***WRONG DESCRIPTION***
        h_RNN_0 = torch.zeros(self.rnn_num_layers, x.size()[0], self.rnn_hidden_size).to(self.device)      #나중에 node 별로 모델 새로 선언 필요
        #initial hidden state 여기서는 zeros, 나중에는 normalized values 시도 | x.size()[0] == batch_size, rnn_hidden_size 정해줘야함
        _, h_RNN_0 = self.rnn(x, h_RNN_0)       #***우리는 여기서 hidden_state가 필요함! 나중에 모든 node에 대해서 hidden_state들을 쌓아야됨***
        #문제는, 여기서의 output값이 필요한지는 확실치 않음 - 이 RNN으로 값을 예측하려는 것이 아니기 때문, 아마 hidden 만 쓸 것으로 생각
        '''
        RNN_hidden_concat = torch.zeros(self.input_size[0], self.rnn_hidden_size).to(self.device)   #10(num_node)*12(num_hidden)
        Conv_hidden_concat = torch.zeros(self.input_size[0], self.conv_channel_num, 2*self.sequence_length - self.input_size[0]).to(self.device)
        
        #여기에 batch 에 따른 iter 필요???
        for batch in range(len(x)):
            x = x[batch]            #batch 하나 가져오기
            
            for i in range(len(self.RNN_modules)):  
                print(x.size())
                single_seq = x[i].reshape([1, 1, self.sequence_length])   #batch 1개 * node 1개 * seq_len // batch 하나에서 RNN을 각각 어떻게 train?
                print(single_seq.size())
                
                #RNN part
                rnn_out_i, _ = self.RNN_modules[f'RNN_{i}'](single_seq)       #RNNs training, activation needed
                RNN_hidden_concat[i] = rnn_out_i       #for future loc-aware-attn? 이렇게 저장하면 안될듯, 나중에 RNN 호출하는걸로
            
                #Dilated Conv part
                conv_out_short_i = self.Dil_Conv_short_modules[f'Dil_Conv_{i}'](single_seq)
                conv_out_long_i = self.Dil_Conv_long_modules[f'Dil_Conv_{i}'](single_seq)
                Conv_hidden_concat[i, :self.sequence_length] = conv_out_short_i
                Conv_hidden_concat[i, self.sequence_length:] = conv_out_long_i
                
                Conv_hidden_concat = torch.relu(Conv_hidden_concat)

        
        
        
        return Conv_hidden_concat
    

2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17


In [ ]:
a = torch.tensor([[[1,2,3,4],[5,6,7,8],[9,10,11,12]]]) #.reshape([1,4])
b = torch.tensor([[[21,22,23,24],[25,26,27,28],[29,30,31,32]]])
#c = torch.stack((a,b), dim=0)
#c = torch.vstack((a, b))
#c = torch.vstack((c, b))
#c
torch.vstack((a,b)).size()

In [ ]:
rnn_hidden_size, seq_len, rnn_num_layers, device

In [ ]:
#Training Part
model = cola_gnn(input_size=X_train.shape[1:],
                 rnn_hidden_size=rnn_hidden_size,
                 sequence_length=seq_len,
                 rnn_num_layers=rnn_num_layers,
                 device=device).to(device)

In [ ]:
loss_graph = []
n= len(trainloader)
for epoch in range(1):
    running_loss = 0.0
    
    for data in trainloader:
        seq, target = data  #batch data
        print(seq.size())
        out = model(seq)
        #loss = criterion(out, target)   #criterion def required
        
        #optimizer.zero_grad()           #optimizer def required
        #loss.backward()
        #optimizer.step()
        #running_loss += loss.item()
        
        print(target)
        
    loss_graph.append(running_loss/n)
    ###print, visualization etc.


In [ ]:
print(model.input_size)

In [ ]:
test = {'rnn0':0, 'rnn1':1, 'rnn2':2}
for i in range(len(test)):
    print(test[f'rnn{i}'])

In [ ]:
#우선 RNN cell을 훈련
num = 0
for i in trainloader:
    print(len(i[0]), num)
    num += 1

class cola_gnn(nn.Module):
    
    def __init__(self, ):
        super().__init__()
            